In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;

#from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC, LinearSVC
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.linear_model import Perceptron
#from sklearn.linear_model import SGDClassifier
#from sklearn.tree import DecisionTreeClassifier
#from xgboost import XGBClassifier
#from sklearn.metrics import precision_score
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Load in the train and test datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Store our passenger ID for easy access
PassengerId = test['PassengerId']

train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
#train,test同步处理
full_data = [train, test]
#give  the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic 
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
#new feature FamilySize
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
for dataset in full_data:
    dataset['isAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1 , 'isAlone'] = 1
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum() # count
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size = age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1) # .group(1) 返回匹配到的title name 
    return ""
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
for dataset in full_data:
    dataset['Sex'] = dataset['Sex'].map({'female':0, 'male':1}).astype(int)
    dataset['Title'] = dataset['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
    dataset['Title'] = dataset['Title'].fillna(0)
    dataset['Embarked'] = dataset['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype(int)
    
      # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

In [4]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [5]:
train.head(3)

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,isAlone,Title
0,0,3,1,1,0,0,0,23,0,2,0,1
1,1,1,0,2,0,3,1,51,1,2,0,3
2,1,3,0,1,0,1,0,22,0,1,1,2


In [6]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction  K-Fold CrossValidation
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [7]:
# out of fold      output crossvalidation
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [8]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 575,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 3
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':575,
    #'max_features': 0.5,
    'max_depth': 5,
    'min_samples_leaf': 3,
    'verbose': 3
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 575,
    'learning_rate' : 0.95
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 575,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 3,
    'verbose': 3
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [9]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [10]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

In [11]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

building tree 1 of 575
building tree 2 of 575building tree 3 of 575building tree 4 of 575building tree 8 of 575



building tree 7 of 575building tree 6 of 575
building tree 5 of 575
building tree 11 of 575
building tree 10 of 575building tree 15 of 575
building tree 9 of 575

building tree 14 of 575
building tree 13 of 575building tree 21 of 575
building tree 12 of 575

building tree 16 of 575
building tree 18 of 575building tree 25 of 575
building tree 17 of 575

building tree 22 of 575
building tree 20 of 575building tree 29 of 575
building tree 19 of 575

building tree 26 of 575
building tree 24 of 575building tree 32 of 575
building tree 23 of 575

building tree 30 of 575
building tree 27 of 575building tree 35 of 575
building tree 28 of 575

building tree 33 of 575
building tree 31 of 575building tree 36 of 575
building tree 34 of 575

building tree 39 of 575
building tree 38 of 575building tree 41 of 575
building tree 37 of 575

building tree 44 of 575
building tree 42 of 575bui

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    0.1s



building tree 160 of 575building tree 177 of 575
building tree 152 of 575

building tree 155 of 575
building tree 162 of 575building tree 181 of 575
building tree 154 of 575

building tree 158 of 575
building tree 166 of 575building tree 186 of 575
building tree 157 of 575

building tree 164 of 575
building tree 170 of 575building tree 189 of 575
building tree 161 of 575

building tree 167 of 575
building tree 174 of 575building tree 194 of 575
building tree 165 of 575

building tree 169 of 575
building tree 179 of 575building tree 198 of 575
building tree 172 of 575

building tree 173 of 575
building tree 184 of 575building tree 200 of 575
building tree 175 of 575

building tree 176 of 575
building tree 187 of 575building tree 206 of 575
building tree 178 of 575
building tree 180 of 575

building tree 210 of 575
building tree 182 of 575building tree 191 of 575
building tree 183 of 575

building tree 215 of 575
building tree 185 of 575building tree 193 of 575
building tree 188 of 575


[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 575 out of 575 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.0s


building tree 369 of 575
building tree 340 of 575building tree 346 of 575
building tree 351 of 575

building tree 372 of 575
building tree 344 of 575building tree 350 of 575
building tree 356 of 575

building tree 375 of 575
building tree 348 of 575building tree 354 of 575
building tree 360 of 575

building tree 379 of 575
building tree 353 of 575building tree 358 of 575
building tree 364 of 575

building tree 383 of 575
building tree 355 of 575building tree 363 of 575
building tree 368 of 575

building tree 385 of 575
building tree 359 of 575building tree 367 of 575
building tree 371 of 575

building tree 390 of 575
building tree 362 of 575building tree 373 of 575
building tree 374 of 575

building tree 394 of 575
building tree 366 of 575building tree 377 of 575
building tree 378 of 575

building tree 400 of 575
building tree 370 of 575building tree 381 of 575
building tree 382 of 575

building tree 405 of 575
building tree 376 of 575building tree 386 of 575
building tree 387 of 575



...

building tree 382 of 575building tree 383 of 575building tree 384 of 575



building tree 386 of 575building tree 385 of 575building tree 388 of 575building tree 387 of 575



building tree 390 of 575building tree 389 of 575building tree 391 of 575


building tree 392 of 575
building tree 395 of 575building tree 393 of 575building tree 394 of 575


building tree 396 of 575
building tree 397 of 575building tree 398 of 575

building tree 399 of 575
building tree 400 of 575building tree 401 of 575

building tree 402 of 575
building tree 403 of 575
building tree 404 of 575
building tree 405 of 575building tree 406 of 575

building tree 407 of 575building tree 408 of 575building tree 409 of 575

building tree 410 of 575

building tree 411 of 575
building tree 413 of 575building tree 412 of 575

building tree 414 of 575building tree 415 of 575

building tree 417 of 575building tree 416 of 575

building tree 418 of 575
building tree 419 of 575
building tree 420 of 575
building tree 422 of 575

[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:    1.0s



building tree 507 of 575building tree 508 of 575building tree 509 of 575building tree 505 of 575



building tree 511 of 575building tree 514 of 575building tree 512 of 575building tree 506 of 575


building tree 515 of 575

building tree 510 of 575building tree 517 of 575building tree 516 of 575


building tree 519 of 575building tree 513 of 575
building tree 520 of 575
building tree 521 of 575
building tree 518 of 575
building tree 523 of 575
building tree 525 of 575
building tree 526 of 575building tree 522 of 575
building tree 524 of 575

building tree 528 of 575
building tree 530 of 575
building tree 527 of 575
building tree 529 of 575
building tree 532 of 575building tree 531 of 575
building tree 533 of 575

building tree 534 of 575
building tree 535 of 575

building tree 536 of 575
building tree 537 of 575
building tree 538 of 575
building tree 539 of 575
building tree 540 of 575
building tree 541 of 575
building tree 542 of 575
building tree 543 of 575
building tree 544 of 575

[Parallel(n_jobs=-1)]: Done 575 out of 575 | elapsed:    1.2s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 575 out of 575 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 575 out of 575 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 575 out of 575 | elapsed:    0

      Iter       Train Loss   Remaining Time 
         1           1.2350           13.15s
         2           1.1474            7.57s
         3           1.0745            5.70s
         4           1.0111            4.77s
         5           0.9587            4.15s
         6           0.9157            3.79s
         7           0.8783            3.48s
         8           0.8448            3.29s
         9           0.8137            3.14s
        10           0.7891            3.02s
        11           0.7679            2.89s
        12           0.7469            2.81s
        13           0.7286            2.74s
        14           0.7124            2.66s
        15           0.6983            2.59s
        16           0.6830            2.53s
        17           0.6713            2.49s
        18           0.6595            2.47s
        19           0.6475            2.44s
        20           0.6368            2.41s
        21           0.6281            2.37s
        2

In [12]:
# feature importance
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)

[ 0.12909613  0.21565529  0.02650687  0.01768231  0.06872618  0.02081879
  0.09422295  0.0677054   0.0622802   0.01224599  0.28505988]
building tree 1 of 575
building tree 2 of 575building tree 3 of 575building tree 4 of 575


building tree 6 of 575building tree 5 of 575
building tree 7 of 575
building tree 10 of 575
building tree 9 of 575building tree 8 of 575
building tree 11 of 575


building tree 12 of 575building tree 13 of 575
building tree 14 of 575building tree 15 of 575
building tree 16 of 575

building tree 17 of 575
building tree 20 of 575building tree 18 of 575
building tree 19 of 575

building tree 22 of 575
building tree 27 of 575building tree 23 of 575
building tree 21 of 575

building tree 26 of 575
building tree 31 of 575building tree 24 of 575
building tree 25 of 575

building tree 29 of 575
building tree 34 of 575building tree 28 of 575
building tree 30 of 575

building tree 33 of 575
building tree 39 of 575building tree 32 of 575
building tree 35 of 575

building tr

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    0.1s


building tree 158 of 575
building tree 151 of 575building tree 149 of 575
building tree 159 of 575

building tree 160 of 575
building tree 155 of 575building tree 153 of 575
building tree 164 of 575

building tree 163 of 575
building tree 161 of 575building tree 156 of 575
building tree 166 of 575

building tree 168 of 575
building tree 167 of 575building tree 162 of 575
building tree 169 of 575

building tree 173 of 575
building tree 172 of 575building tree 165 of 575
building tree 171 of 575

building tree 176 of 575
building tree 178 of 575building tree 170 of 575
building tree 175 of 575

building tree 180 of 575
building tree 181 of 575building tree 174 of 575
building tree 179 of 575

building tree 184 of 575
building tree 186 of 575building tree 177 of 575building tree 183 of 575



building tree 189 of 575building tree 182 of 575building tree 191 of 575building tree 187 of 575



building tree 192 of 575building tree 185 of 575building tree 193 of 575building tree 190 of 575




[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    0.3s


building tree 358 of 575



building tree 360 of 575building tree 359 of 575building tree 357 of 575building tree 361 of 575



building tree 366 of 575building tree 362 of 575building tree 364 of 575
building tree 363 of 575

building tree 369 of 575
building tree 365 of 575building tree 367 of 575
building tree 368 of 575

building tree 374 of 575
building tree 370 of 575building tree 372 of 575
building tree 371 of 575

building tree 379 of 575
building tree 373 of 575building tree 376 of 575
building tree 375 of 575

building tree 384 of 575
building tree 377 of 575building tree 380 of 575
building tree 378 of 575

building tree 388 of 575
building tree 381 of 575building tree 385 of 575
building tree 382 of 575


building tree 386 of 575building tree 389 of 575building tree 383 of 575


building tree 390 of 575building tree 394 of 575building tree 387 of 575


building tree 393 of 575building tree 396 of 575building tree 391 of 575


building tree 397 of 575building tree 395 of 57

[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 575 out of 575 | elapsed:    0.7s finished


building tree 523 of 575

building tree 518 of 575
building tree 510 of 575building tree 513 of 575
building tree 527 of 575

building tree 521 of 575
building tree 514 of 575building tree 517 of 575
building tree 531 of 575

building tree 526 of 575
building tree 516 of 575building tree 520 of 575

building tree 535 of 575
building tree 525 of 575building tree 522 of 575
building tree 530 of 575


building tree 529 of 575building tree 524 of 575building tree 534 of 575

building tree 541 of 575
building tree 532 of 575building tree 528 of 575
building tree 537 of 575


building tree 536 of 575building tree 533 of 575building tree 547 of 575building tree 540 of 575



building tree 539 of 575building tree 538 of 575building tree 551 of 575building tree 543 of 575



building tree 544 of 575building tree 542 of 575building tree 548 of 575

building tree 555 of 575
building tree 546 of 575building tree 545 of 575
building tree 552 of 575


building tree 550 of 575building tree 549 of 575

In [13]:
rf_features = [0.10474135,  0.21837029,  0.04432652,  0.02249159,  0.05432591,  0.02854371
  ,0.07570305,  0.01088129 , 0.24247496,  0.13685733 , 0.06128402]
et_features = [ 0.12165657,  0.37098307  ,0.03129623 , 0.01591611 , 0.05525811 , 0.028157
  ,0.04589793 , 0.02030357 , 0.17289562 , 0.04853517,  0.08910063]
ada_features = [0.028 ,   0.008  ,      0.012   ,     0.05866667,   0.032 ,       0.008
  ,0.04666667 ,  0.     ,      0.05733333,   0.73866667,   0.01066667]
gb_features = [ 0.06796144 , 0.03889349 , 0.07237845 , 0.02628645 , 0.11194395,  0.04778854
  ,0.05965792 , 0.02774745,  0.07462718,  0.4593142 ,  0.01340093]

In [14]:
cols = train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_features,
     'Extra Trees  feature importances': et_features,
      'AdaBoost feature importances': ada_features,
    'Gradient Boost feature importances': gb_features
    })

In [16]:
# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Random Forest feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Random Forest feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Extra Trees  feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Extra Trees  feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Extra Trees Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['AdaBoost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['AdaBoost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Gradient Boost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Gradient Boost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [15]:
# Create the new column containing the average of values

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.head(3)

,AdaBoost feature importances,Extra Trees feature importances,Gradient Boost feature importances,Random Forest feature importances,features,mean
0,0.028,0.121657,0.067961,0.104741,Pclass,0.080590
1,0.008,0.370983,0.038893,0.218370,Sex,0.159062
2,0.012,0.031296,0.072378,0.044327,Age,0.040000


In [18]:
y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')


In [16]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

,AdaBoost,ExtraTrees,GradientBoost,RandomForest
0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0


In [17]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

In [18]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 5000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=1,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)

In [19]:
# Generate Submission File 
Submission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })
Submission.to_csv("Submission with new parameters.csv", index=False)